In [1]:
require('nn')
require('hdf5')
require('optim')

In [2]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

sentences = 42068

window_size = X_train:size(2)
n_actual = nfeatures

In [165]:
function bigramCount(X, K)
    local countMatrix = torch.zeros(K, K)
    for i=1,X:size(1) do
        for j=2,X:size(2) do
            local prev_word = X[i][j-1]
            local curr_word = X[i][j]
            countMatrix[prev_word][curr_word] = countMatrix[prev_word][curr_word] + 1
        end
    end
    return countMatrix
end

function unigramCount(X, Y, K)
    local countVector = torch.zeros(1, K)
    for i=1,X:size(1) do
        for j=2,X:size(2) do
            local idx = X[i][j]
            countVector[1][idx] = countVector[1][idx] + 1
        end
    end
    return countVector
end

In [147]:
function additive(matrix, alpha)
    local add_matrix = matrix:add(alpha)
    return add_matrix
end

function normalize(CV, CM)
    --divide by counts of previous word
    local norm = torch.cdiv(CM,torch.expand(CV:t(), CM:size(1), CM:size(2)))
    return norm
end

function bigramPerp(bcm, X, correct, q_50)
    p = 0
    for i=1,X:size(1) do
        idx = q_50[i][correct[i]]
        prev_word = X[i][5]
        p = torch.log(bcm[prev_word][idx]) + p
    end
    return torch.exp(-p/X_valid:size(1))
end

function kaggle(bcm, test, q)
    scores = torch.zeros(q:size(1), 50)
    for i=1,test:size(1) do
        prev = test[i][5]
        for j=1,50 do
            w = q[i][j]
            scores[i][j] = bcm[prev][w]
        end
    end
    return scores:cdiv(torch.expand((torch.sum(scores, 2)), scores:size(1), scores:size(2)))
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [152]:
CM = bigramCount(X_train, nfeatures)

In [153]:
CV = unigramCount(X_train, Y_train, nfeatures)

In [154]:
BCM = normalize(CV, CM)

In [156]:
scores = kaggle(BCM, X_test, q_test)

In [157]:
write2file(scores, "asdf.csv")

In [167]:
bigramPerp(bcm, X_valid, kaggle_valid, q_valid)

1268.7793631882	


In [166]:
bcm = torch.cdiv(BCM, torch.expand(BCM:sum(2), BCM:size(1), BCM:size(2)))

In [146]:
bcm:sub(1,1):sum()

0.99999999999975	


In [9]:
X_train:sub(1,50)

     3      3      3   9981  10001
     3      3   9981  10001   9987
     3   9981  10001   9987   9978
  9981  10001   9987   9978   9991
 10001   9987   9978   9991   9975
  9987   9978   9991   9975   9985
  9978   9991   9975   9985   9977
  9991   9975   9985   9977   9993
  9975   9985   9977   9993   9972
  9985   9977   9993   9972   9973
  9977   9993   9972   9973   9989
  9993   9972   9973   9989   9997
  9972   9973   9989   9997   9979
  9973   9989   9997   9979   9983
  9989   9997   9979   9983   9974
  9997   9979   9983   9974   9984
  9979   9983   9974   9984   9990
  9983   9974   9984   9990   9976
  9974   9984   9990   9976  10000
  9984   9990   9976  10000   9994
  9990   9976  10000   9994   9995
  9976  10000   9994   9995   9998
 10000   9994   9995   9998   9986
     3      3      3   9000      2
     3      3   9000      2      5
     3   9000      2      5     74
  9000      2      5     74    395
     2      5     74    395     35
     5     74    395

In [3]:
new_sen_idx = torch.zeros(1, 41931)
sen_count = 0
for i=1,X_train:size(1) do
    row = X_train[i]
    if (row[1] ==row[2] and row[2]==row[3] and row[1]==3) then
        sen_count = sen_count + 1
        new_sen_idx[1][sen_count] = i 
    end
end

In [35]:
sen_counter = 2
i = 1
while i < 50 do
    if i <= new_sen_idx[1][sen_counter] then
        print(test[i])
        i = i + 5
    else
        last_win = new_sen_idx[1][sen_counter]-1
        print(i-5)
        print(new_sen_idx[1][sen_counter]-21)
        print(test[last_win])
        i = new_sen_idx[1][sen_counter]
        sen_counter = sen_counter + 1
    end
end

     3
     3
     3
  9981
 10001
[torch.LongTensor of size 5]

 9987
 9978
 9991
 9975
 9985
[torch.LongTensor of size 5]

 9977
 9993
 9972
 9973
 9989
[torch.LongTensor of size 5]

 9997
 9979
 9983
 9974
 9984
[torch.LongTensor of size 5]

  9990
  9976
 10000
  9994
  9995
[torch.LongTensor of size 5]

21	
3	
 10000
  9994
  9995
  9998
  9986
[torch.LongTensor of size 5]

    3
    3
    3
 9000
    2
[torch.LongTensor of size 5]

    5
   74
  395
   35
 2150
[torch.LongTensor of size 5]

    1
  148
   21
    8
 9477
[torch.LongTensor of size 5]

34	
17	
    8
 9477
  277
  409
    5
[torch.LongTensor of size 5]

  3
  3
  3
 25
  2
[torch.LongTensor of size 5]

   15
  144
    6
    2
 5461
[torch.LongTensor of size 5]

    3
    3
    3
 7254
    2
[torch.LongTensor of size 5]



In [5]:
test = X_train:sub(1,50)

In [14]:
test

     3      3      3   9981  10001
     3      3   9981  10001   9987
     3   9981  10001   9987   9978
  9981  10001   9987   9978   9991
 10001   9987   9978   9991   9975
  9987   9978   9991   9975   9985
  9978   9991   9975   9985   9977
  9991   9975   9985   9977   9993
  9975   9985   9977   9993   9972
  9985   9977   9993   9972   9973
  9977   9993   9972   9973   9989
  9993   9972   9973   9989   9997
  9972   9973   9989   9997   9979
  9973   9989   9997   9979   9983
  9989   9997   9979   9983   9974
  9997   9979   9983   9974   9984
  9979   9983   9974   9984   9990
  9983   9974   9984   9990   9976
  9974   9984   9990   9976  10000
  9984   9990   9976  10000   9994
  9990   9976  10000   9994   9995
  9976  10000   9994   9995   9998
 10000   9994   9995   9998   9986
     3      3      3   9000      2
     3      3   9000      2      5
     3   9000      2      5     74
  9000      2      5     74    395
     2      5     74    395     35
     5     74    395